<a href="https://colab.research.google.com/github/entangelk/study_AIs/blob/main/docs/quests/MLs/SpineSurgeryList_GridSearchCV_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df_SSL = pd.read_csv('SpineSurgeryList.csv')
df_SSL[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [3]:
df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '혈액형', '직업', '재발여부' ]]
# df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '재발여부' ]]
df_SSL_extract[:2]

,성별,신장,체중,흡연여부,연령,혈액형,직업,재발여부
0,2,163,60.3,0,66,RH+A,자영업,0
1,1,171,71.7,0,47,RH+A,운동선수,0


In [4]:
df_SSL_extract.isnull().sum()

성별        0
신장        0
체중        0
흡연여부      0
연령        0
혈액형       0
직업      415
재발여부      0
dtype: int64

### 전처리

In [5]:
df_SSL_extract.dropna(inplace=True)

<ipython-input-5-8f370ca9152c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SSL_extract.dropna(inplace=True)


In [6]:
target = df_SSL_extract['재발여부']
Xs = df_SSL_extract.drop(columns=['재발여부'])
features = pd.get_dummies(Xs) # 범주형을 원핫인코딩으로 바꿔주기는하는데 쓰지말래 왜? 몰라 근데 보니까 숫자로 된 범주형은 안바뀌었네. 이걸 어차피 바꿔줘야하니까 꼭 쓸필요는 없을것같다.
features[:2]

,성별,신장,체중,흡연여부,연령,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,직업_건설업,...,직업_사무직,직업_사업가,직업_예술가,직업_운동선수,직업_운수업,직업_의료직,직업_자영업,직업_주부,직업_특수전문직,직업_학생
0,2,163,60.3,0,66,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,171,71.7,0,47,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## resampling

In [7]:
from collections import Counter
Counter(target)
# 타겟의 비율 확인
# 0에 치우처져 있음

Counter({0: 1302, 1: 177})

### Oversampling (많은 갯수 기준으로 맞춤)
- 적은 숫자에 대한  테스트 케이스를 더 넣어줘서 많은 갯수에 가깝게 맞춰줌

In [8]:
from imblearn.over_sampling import SMOTE
overSampling = SMOTE(sampling_strategy=0.8)
feature_oversample, target_oversample =  overSampling.fit_resample(features,target)
feature_oversample.shape, target_oversample.shape

((2343, 26), (2343,))

In [9]:
Counter(target_oversample)

Counter({0: 1302, 1: 1041})

### UnderSampling(적은 갯수 기준으로 맞춤)

In [10]:
from imblearn.under_sampling import NearMiss
underSampling = NearMiss(sampling_strategy=0.8)
feature_undersample, target_undersample = underSampling.fit_resample(features,target)
feature_undersample.shape, target_undersample.shape

((398, 26), (398,))

In [11]:
Counter(target_undersample)

Counter({0: 221, 1: 177})

### over와 under 결합 sampling

In [12]:
from imblearn.combine import SMOTEENN
combineSampling = SMOTEENN(sampling_strategy=0.5)
feature_combinesample, target_combinesample = combineSampling.fit_resample(features,target)
feature_combinesample.shape, target_combinesample.shape

((1182, 26), (1182,))

In [13]:
Counter(target_combinesample)

Counter({0: 792, 1: 390})

## 특성 공학 - Cross Validation

In [15]:
# 모델 : hyper parameters 교차 적용, 데이터셋 : 알맞게 섞어서 대입해줌
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
hyper_parameters = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
estimator_model = DecisionTreeClassifier()
# grid_model = GridSearchCV(estimator_model,hyper_parameters)

# score 방식 지정
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)
grid_model = GridSearchCV(estimator_model,hyper_parameters,scoring=scoring,cv = 5)

In [16]:
grid_model

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

### 각 샘플링에 대한 모델 적용

#### Oversampling 모델 적용

In [17]:
grid_model.fit(feature_oversample, target_oversample)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [18]:
over_best_model = grid_model.best_estimator_

#### undersampling 모델 적용

In [19]:
grid_model.fit(feature_undersample, target_undersample)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [20]:
under_best_model = grid_model.best_estimator_

#### combinesampling 모델 적용

In [21]:
grid_model.fit(feature_combinesample, target_combinesample)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [22]:
combine_best_model = grid_model.best_estimator_

## 평가

In [23]:
from sklearn.metrics import classification_report

#### oversampling 모델 평가

In [28]:
target_predict = over_best_model.predict(features)
print(classification_report(target, target_predict))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1302
           1       0.52      0.50      0.51       177

    accuracy                           0.88      1479
   macro avg       0.72      0.72      0.72      1479
weighted avg       0.88      0.88      0.88      1479



#### undersampling 모델 평가

In [30]:
target_predict = under_best_model.predict(features)
print(classification_report(target, target_predict))

              precision    recall  f1-score   support

           0       0.91      0.61      0.73      1302
           1       0.17      0.58      0.26       177

    accuracy                           0.61      1479
   macro avg       0.54      0.59      0.49      1479
weighted avg       0.82      0.61      0.67      1479



#### combinesampling 모델 평가

In [32]:
target_predict = combine_best_model.predict(features)
print(classification_report(target, target_predict))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90      1302
           1       0.36      0.54      0.43       177

    accuracy                           0.83      1479
   macro avg       0.65      0.70      0.66      1479
weighted avg       0.86      0.83      0.84      1479



##결론
 + f1 score가 가장 높으면서, 타겟의 군집간 스코어의 차이가 작고 가중 평균치가 가장 높은 oversampling으로 학습한 모델이 더 성능이 좋다고 판단된다.